In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import pandas as pd
import numpy as np
import subprocess
import os
import pathlib
import numpy as np
import u19_pipeline.utils.path_utils as pu
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from shutil import which

import u19_pipeline.utils.matlab_utils as mu
from scipy.io import loadmat, matlab

acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')
from u19_pipeline import lab


Connecting alvaros@datajoint00.pni.princeton.edu:3306


## Add MATLAB path to system PATH 

In [3]:
local_matlab_path = '/Applications/MATLAB_R2020a.app/bin/'
if which('matlab') is None:
    os.environ['PATH'] += ':'+local_matlab_path

In [3]:
if which('matlab') is None:
    print('Matlab command not found, add correct path installation or proceed (Choice &  TrialType data will not be read from the behavior file')
else:
    print('Matlab installation found !!')

Matlab command not found, add correct path installation or proceed (Choice &  TrialType data will not be read from the behavior file


In [4]:
subject = 'efonseca_ef884_actp003'
date = '2021-12-03'

path = (acquisition.SessionStarted & dict(session_date =date, subject_fullname=subject)).fetch1('new_remote_path_behavior_file')


mat_file = lab.Path().get_local_path2(path)
mat_file

PosixPath('/Volumes/braininit/Data/Raw/behavior/efonseca/efonseca_ef884_actp003/20211203_g0/LSTT_Levels_FirstMemory_EF_LSTT_Memory_165A-Rig8-T_efonseca_ef884_actp003_T_20211203.mat')

In [5]:
df_behavior_file = mu.convert_behavior_file(mat_file)
df_behavior_file

A working matlab version was not found. Code version, animal protocol, type of trial, and choice could not be saved to NWB.
{}


KeyError: 'shaping_protocol'

In [17]:
df_behavior_file.columns

Index(['block', 'trial_idx', 'choice', 'collision', 'comment', 'cueCombo',
       'cueOffset', 'cueOnset', 'cuePos', 'duration_block', 'excessTravel',
       'finalchoice', 'iArmEntry', 'iBlank', 'iCueEntry', 'iMemEntry',
       'iTurnEntry', 'iterations', 'licks', 'position', 'rewardScale',
       'sensorDots', 'start_block', 'state_ports', 'time', 'trialID',
       'trialProb', 'trialType', 'velocity', 'viStart', 'versionIndex',
       'firstTrial', 'start_trial', 'duration_trial', 'rewardMiL', 'mazeID',
       'mainMazeID', 'motionBlurRange', 'iterStr', 'shapingProtocol',
       'frozenStimuli', 'stimulusBank', 'stimulusCommit', 'stimulusConfig',
       'stimulusSet', 'easyBlockFlag'],
      dtype='object')